In [1]:
import sys
import os
os.chdir('/scratch/nhl256/dl_project/code/')

import random
import argparse

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from data_helper import LabeledDataset
from helper import *

import math
import time

import utils

In [2]:
image_folder = 'data/data'
annotation_csv = 'data/data/annotation.csv'


cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
print(device)

cuda:0


In [3]:
def get_transform(): 
    return torchvision.transforms.ToTensor()

In [4]:
labeled_scene_index = np.arange(120, 134)
labeled_trainset = LabeledDataset(
    image_folder=image_folder,
    annotation_file=annotation_csv,
    scene_index=labeled_scene_index,
    transform=get_transform(),
    extra_info=False
    )
dataloader = torch.utils.data.DataLoader(
    labeled_trainset,
    batch_size=1,
    shuffle=False,
    num_workers=4
    )

### fastRCNN - Version of 05/04/20

- Concat 6 images along 1 dim before passing through the model
- Saving the feature extractor and the fasterRCNN model
- ResNet50 for feature extraction using the weights from self-supervised task (/scratch/bva212/dl_project/jigsaw_best_model_fe.pth)
- optimizer = torch.optim.Adam(params, lr=0.0001)

In [5]:
def get_transform(): 
    return torchvision.transforms.ToTensor()

def reorder_coord(pred_bboxes):
    xmin, ymin, xmax, ymax = pred_bboxes.unbind(1)
    return torch.stack((xmax, xmax, xmin, xmin, ymax, ymin, ymax, ymin), dim=1).view(-1, 2, 4)

def concat_features(features, dim = 2):
    #dim 0 ==> stacking the images in the channel dimension
    #dim 1 ==> stacking the images in row dimension
    #dim 2 ==> stacking the images in column dimension
    tensor_tuples = torch.unbind(features, dim=0)
    concatenated_fm = torch.cat(tensor_tuples, dim=dim)
    return concatenated_fm 

In [6]:
model_path = '/scratch/nhl256/dl_project/model/object_detection_resnet50_0504_epoch9.pth'
checkpoint = torch.load(model_path)
feature_extractor_weights = checkpoint['feature_extractor']
model_weights = checkpoint['fasterRCNN']

In [8]:
feature_extractor_weights = checkpoint['feature_extractor']
model_weights = checkpoint['fasterRCNN']

# Initiate the feature extractor
# Get model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
model = model.to(device)
model.load_state_dict(model_weights)

# Get feature extractor
feature_extractor = torchvision.models.resnet50(pretrained=False)
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-2])
feature_extractor.load_state_dict(torch.load('/scratch/bva212/dl_project/jigsaw_best_model_fe.pth'))
feature_extractor.to(device)

model.eval()
feature_extractor.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [9]:
def get_bounding_boxes(samples):
    # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
    # You need to return a tuple with size 'batch_size' and each element is a cuda tensor [N, 2, 4]
    # where N is the number of object

    #Preparing inputs
    batchsize = samples.shape[0]
    fe_batch = []
    for i in range(batchsize):
        image_tensor = sample[i]
        features = feature_extractor(image_tensor)
        #print(features.shape)
        features = concat_features(features)
        features = features.view(3, 2048, 160)
        #print(features.shape)
        fe_batch.append(features)

    images = list(image.to(device) for image in fe_batch)
    predictions = model(images)
    res = []
    for i in range(len(predictions)):
        prediction = predictions[i]
        pred_bboxes = prediction['boxes']
        reorder_pred_bboxes = reorder_coord(pred_bboxes)
        res.append(reorder_pred_bboxes)

    return res

In [10]:
total = 0
total_ats_bounding_boxes = 0

for i, data in enumerate(dataloader):
    total += 1
    sample, target, road_image = data
    sample = sample.cuda()

    predicted_bounding_boxes = get_bounding_boxes(sample)[0].cpu()
    

    ats_bounding_boxes = compute_ats_bounding_boxes(predicted_bounding_boxes,
                                                    target['bounding_box'][0])
    print('Number of pred bboxes {}'.format(predicted_bounding_boxes.shape))
    print('ats_bounding_boxes {}'.format(ats_bounding_boxes))

    total_ats_bounding_boxes += ats_bounding_boxes


Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats

Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.011979506351053715
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.003271070308983326
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.005924709606915712
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0061468868516385555
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.011979506351053715
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.003271070308983326
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.003311454085633

Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats

Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats

Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.006311242002993822
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.013798416592180729
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.017270972952246666
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0033114540856331587
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.006311242002993822
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.018406260758638382
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.015016959048807621
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.010058541782200336
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0032316597644239664
Number of pred bboxes torch.S

...

Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0029155192896723747
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0055252909660339355
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0029803086072206497
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxe

Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0034388175699859858
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0034388175699859858
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.003395288484171033
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.006304499227553606
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0034388175699859858
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.

Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0033114540856331587
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.003271070308983326
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.003271070308983326
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.003271070308983326
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0032316597644239664
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0059969620779156685
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0032316597644239664
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_box

Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0072695971466600895
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0072695971466600895
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.002853486919775605
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0069024465046823025
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.007892441004514694
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.007892441004514694
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.007653974462300539
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.005017866380512714
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.014792666770517826
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.005231392569839954
Number of p

Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.003944525960832834
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.007126824930310249
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.007126824930310249
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0

In [11]:
print(total_ats_bounding_boxes)

tensor(4.3190)


## fastRCNN - Version of 05/02

- Resnet18
- PosConcat

In [5]:
def get_transform(): 
    return torchvision.transforms.ToTensor()

def reorder_coord(pred_bboxes):
    xmin, ymin, xmax, ymax = pred_bboxes.unbind(1)
    return torch.stack((xmax, xmax, xmin, xmin, ymax, ymin, ymax, ymin), dim=1).view(-1, 2, 4)

def position_concat_features(feature_maps, dim=2):
    '''
    feature maps be in shape V C H W
    returns in shape C H W
    '''
    first_row = torch.cat([feature_maps[i] for i in range(3)], dim=dim)
    second_row = torch.cat([feature_maps[i] for i in range(3,6)], dim=dim)
    result = torch.cat([first_row, second_row], dim=1)
    return result

In [6]:
model_path = '/scratch/nhl256/dl_project/model/object_detection_resnet18_poscat_0502_epoch3.pth'
checkpoint = torch.load(model_path)
feature_extractor_weights = checkpoint['feature_extractor']
model_weights = checkpoint['fasterRCNN']

In [8]:
feature_extractor_weights = checkpoint['feature_extractor']
model_weights = checkpoint['fasterRCNN']

# Initiate the feature extractor
# Get model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
model = model.to(device)
model.load_state_dict(model_weights)

# Get feature extractor
feature_extractor = torchvision.models.resnet18(pretrained=False)
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-2])
feature_extractor.to(device)
feature_extractor.load_state_dict(feature_extractor_weights)

model.eval()
feature_extractor.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [11]:
def get_bounding_boxes(samples):
    # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
    # You need to return a tuple with size 'batch_size' and each element is a cuda tensor [N, 2, 4]
    # where N is the number of object

    #Preparing inputs
    batchsize = samples.shape[0]
    fe_batch = []
    for i in range(batchsize):
        image_tensor = sample[i]
        features = feature_extractor(image_tensor)
        #print(features.shape)
        features = position_concat_features(features)
        features = features.view(3, 512, 160)
        #print(features.shape)
        fe_batch.append(features)

    images = list(image.to(device) for image in fe_batch)
    predictions = model(images)
    res = []
    for i in range(len(predictions)):
        prediction = predictions[i]
        pred_bboxes = prediction['boxes']
        reorder_pred_bboxes = reorder_coord(pred_bboxes)
        res.append(reorder_pred_bboxes)

    return res

In [12]:
total = 0
total_ats_bounding_boxes = 0
total_ts_road_map = 0
for i, data in enumerate(dataloader):
    total += 1
    sample, target, road_image = data
    sample = sample.cuda()

    predicted_bounding_boxes = get_bounding_boxes(sample)[0].cpu()
    

    ats_bounding_boxes = compute_ats_bounding_boxes(predicted_bounding_boxes,
                                                    target['bounding_box'][0])
    print('Number of pred bboxes {}'.format(predicted_bounding_boxes.shape))
    print('ats_bounding_boxes {}'.format(ats_bounding_boxes))

    total_ats_bounding_boxes += ats_bounding_boxes


Number of pred bboxes torch.Size([77, 2, 4])
ats_bounding_boxes 0.008333440870046616
Number of pred bboxes torch.Size([72, 2, 4])
ats_bounding_boxes 0.006470407359302044
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.004728374537080526
Number of pred bboxes torch.Size([93, 2, 4])
ats_bounding_boxes 0.00273701804690063
Number of pred bboxes torch.Size([68, 2, 4])
ats_bounding_boxes 0.006645282730460167
Number of pred bboxes torch.Size([79, 2, 4])
ats_bounding_boxes 0.01013076864182949
Number of pred bboxes torch.Size([67, 2, 4])
ats_bounding_boxes 0.003831825451925397
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.010512957349419594
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.004126581363379955
Number of pred bboxes torch.Size([70, 2, 4])
ats_bounding_boxes 0.006736313924193382
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.007231631316244602
Number of pred bboxes torch.Size([69, 2, 4])
ats_bounding_boxes 0.

Number of pred bboxes torch.Size([74, 2, 4])
ats_bounding_boxes 0.0029475579503923655
Number of pred bboxes torch.Size([77, 2, 4])
ats_bounding_boxes 0.008234512992203236
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.003271070308983326
Number of pred bboxes torch.Size([75, 2, 4])
ats_bounding_boxes 0.0053451182320714
Number of pred bboxes torch.Size([85, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([69, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.004239232279360294
Number of pred bboxes torch.Size([99, 2, 4])
ats_bounding_boxes 0.00898555014282465
Number of pred bboxes torch.Size([84, 2, 4])
ats_bounding_boxes 0.016767427325248718
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.003271070308983326
Number of pred bboxes torch.Size([77, 2, 4])
ats_bounding_boxes 0.01786690019071102
Number of pred bboxes torch.Size([91, 2, 4])
ats_bounding_boxes 0.009752651676535606
Number of pred b

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.005840532016009092
Number of pred bboxes torch.Size([98, 2, 4])
ats_bounding_boxes 0.004390633665025234
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.004239232279360294
Number of pred bboxes torch.Size([97, 2, 4])
ats_bounding_boxes 0.00615623639896512
Number of pred bboxes torch.Size([88, 2, 4])
ats_bounding_boxes 0.002682277699932456
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.002373697003349662
Number of pred bboxes torch.Size([92, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([93, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([87, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([89, 2, 4])
ats_bounding_boxes 0.0026296840514987707
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_bo

Number of pred bboxes torch.Size([91, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([74, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([93, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([76, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([91, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([80, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([79, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([67, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([68, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([78, 2, 4])
ats_bounding_boxes 0.005652309395372868
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([73, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Si

Number of pred bboxes torch.Size([70, 2, 4])
ats_bounding_boxes 0.008760797791182995
Number of pred bboxes torch.Size([97, 2, 4])
ats_bounding_boxes 0.01314119715243578
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.008541777729988098
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.003777856007218361
Number of pred bboxes torch.Size([86, 2, 4])
ats_bounding_boxes 0.002853486919775605
Number of pred bboxes torch.Size([74, 2, 4])
ats_bounding_boxes 0.005854177754372358
Number of pred bboxes torch.Size([85, 2, 4])
ats_bounding_boxes 0.007892441004514694
Number of pred bboxes torch.Size([92, 2, 4])
ats_bounding_boxes 0.014460000209510326
Number of pred bboxes torch.Size([90, 2, 4])
ats_bounding_boxes 0.00885725487023592
Number of pred bboxes torch.Size([79, 2, 4])
ats_bounding_boxes 0.005176325794309378
Number of pred bboxes torch.Size([78, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([93, 2, 4])
ats_bounding_boxes 0.011122185736894608

...

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.00235287519171834
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.004747394006699324
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.008243403397500515
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0022925452794879675
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.008907700888812542
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0022925452794879675
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.01085109356790781
Number of pred bboxes torch.Size([100, 2, 4])
ats_bou

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.005942105781286955
Number of pred bboxes torch.Size([79, 2, 4])
ats_bounding_boxes 0.01171728502959013
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.019599586725234985
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.010203838348388672
Number of pred bboxes torch.Size([90, 2, 4])
ats_bounding_boxes 0.02129332534968853
Number of pred bboxes torch.Size([89, 2, 4])
ats_bounding_boxes 0.02209528535604477
Number of pred bboxes torch.Size([61, 2, 4])
ats_bounding_boxes 0.0186070017516613
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.012381489388644695
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.009523707441985607
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.02462778426706791
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0158563069999218
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.009698533453047276
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.009314163587987423
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.01140323281288147
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.01140323281288147
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.005970333702862263
Number of pred bboxes torch.Size([86, 2, 4])
ats_bounding_boxes 0.0046648308634757996
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.008509905077517033
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.014178307726979256
Number of pred bboxes torch.Size([79, 2, 4])
ats_bounding_boxes 0.008781267330050468
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.008959095925092697
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.006398379802703857
Number of pred bboxes torch.Size([100, 2, 4])
ats_boundin

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.012488625012338161
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.011548836715519428
Number of pred bboxes torch.Size([82, 2, 4])
ats_bounding_boxes 0.01846078410744667
Number of pred bboxes torch.Size([90, 2, 4])
ats_bounding_boxes 0.015691472217440605
Number of pred bboxes torch.Size([92, 2, 4])
ats_bounding_boxes 0.009396137669682503
Number of pred bboxes torch.Size([86, 2, 4])
ats_bounding_boxes 0.010407228954136372
Number of pred bboxes torch.Size([89, 2, 4])
ats_bounding_boxes 0.005176325794309378
Number of pred bboxes torch.Size([75, 2, 4])
ats_bounding_boxes 0.014659097418189049
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.009961523115634918
Number of pred bboxes torch.Size([75, 2, 4])
ats_bounding_boxes 0.0061468868516385555
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.007774718105792999
Number of pred bboxes torch.Size([74, 2, 4])
ats_bounding_boxes

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.004430188797414303
Number of pred bboxes torch.Size([59, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([77, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([75, 2, 4])
ats_bounding_boxes 0.005652309395372868
Number of pred bboxes torch.Size([85, 2, 4])
ats_bounding_boxes 0.010462785139679909
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.004430188797414303
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.004430188797414303
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.006953638978302479
Number of pred bboxes torch.Size([90, 2, 4])
ats_bounding_boxes 0.0026557205710560083
Number of pred bboxes torch.Size([96, 2, 4])
ats_bounding_boxes 0.005109100602567196
Number of pred bboxes torch.Size([86, 2, 4])
ats_bounding_boxes 0.00512240594252944
Number of pred bboxes torch.Size([87, 2, 4])
ats_bounding_boxes 0.005069596692919731
Number of 

In [13]:
print(total_ats_bounding_boxes)

tensor(15.9355)


## fastRCNN - Version of 05/02

- Resnet18
- Concat 6 images along 1 dim

In [5]:
def get_transform(): 
    return torchvision.transforms.ToTensor()

def reorder_coord(pred_bboxes):
    xmin, ymin, xmax, ymax = pred_bboxes.unbind(1)
    return torch.stack((xmax, xmax, xmin, xmin, ymax, ymin, ymax, ymin), dim=1).view(-1, 2, 4)

def concat_features(features, dim = 2):
    #dim 0 ==> stacking the images in the channel dimension
    #dim 1 ==> stacking the images in row dimension
    #dim 2 ==> stacking the images in column dimension
    tensor_tuples = torch.unbind(features, dim=0)
    concatenated_fm = torch.cat(tensor_tuples, dim=dim)
    return concatenated_fm 

In [17]:
# THIS IS NOT WORKING NEED TO FIX

# class ModelLoader():
#     # Fill the information for your team
#     team_name = 'team_name'
#     round_number = 1
#     team_member = []
#     contact_email = '@nyu.edu'

#     def __init__(self, model_file='put_your_model_file_name_here'):
#         """
#         model_file = {'get_bboxes_model': object_detection_model_path,
#                       'get_binary_RM_model': None}
#         """
# #         model_path = '/scratch/nhl256/dl_project/model/object_detection_resnet18_0502_epoch7.pth'
# #         checkpoint = torch.load(model_path)
# #         feature_extractor_weights = checkpoint['feature_extractor']
# #         model_weights = checkpoint['fasterRCNN']
        
#         self.bboxes_model_file =  model_file['get_bboxes_model']
#         self.bboxes_checkpoint = torch.load(self.bboxes_model_file)
        
#         self.bboxes_feature_extractor_weights = self.bboxes_checkpoint['feature_extractor']
#         self.bboxes_model_weights = self.bboxes_checkpoint['fasterRCNN']
        
        
#         # Initiate the feature extractor
#         # Get model
#         self.bboxes_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
#         self.bboxes_model = self.bboxes_model.to(device)
#         self.bboxes_model.load_state_dict(self.bboxes_model_weights)

#         # Get feature extractor
#         self.bboxes_feature_extractor = torchvision.models.resnet18(pretrained=False)
#         self.bboxes_feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-2])
#         self.bboxes_feature_extractor.to(device)
#         self.bboxes_feature_extractor.load_state_dict(self.bboxes_feature_extractor_weights)

#         self.bboxes_model.eval()
#         self.bboxes_feature_extractor.eval()
    
    
#     def get_bounding_boxes(self, samples):
#         # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
#         # You need to return a tuple with size 'batch_size' and each element is a cuda tensor [N, 2, 4]
#         # where N is the number of object

#         #Preparing inputs
#         batchsize = samples.shape[0]
#         fe_batch = []
#         for i in range(batchsize):
#             image_tensor = sample[i]
#             features = self.bboxes_feature_extractor(image_tensor)
#             #print(features.shape)
#             features = concat_features(features)
#             features = features.view(3, 512, 160)
#             #print(features.shape)
#             fe_batch.append(features)

#         images = list(image.to(device) for image in fe_batch)
#         predictions = self.bboxes_model(images)
#         res = []
#         for i in range(len(predictions)):
#             prediction = predictions[i]
#             pred_bboxes = prediction['boxes']
#             reorder_pred_bboxes = reorder_coord(pred_bboxes)
#             res.append(reorder_pred_bboxes)

#         return res

#     def get_binary_road_map(self, samples):
#         # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
#         # You need to return a cuda tensor with size [batch_size, 800, 800] 
        
#         return torch.rand(1, 800, 800) > 0.5

In [26]:
model_path = '/scratch/nhl256/dl_project/model/object_detection_resnet18_0502_epoch7.pth'
checkpoint = torch.load(model_path)
feature_extractor_weights = checkpoint['feature_extractor']
model_weights = checkpoint['fasterRCNN']

In [27]:
feature_extractor_weights = checkpoint['feature_extractor']
model_weights = checkpoint['fasterRCNN']

In [28]:
# Initiate the feature extractor
# Get model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
model = model.to(device)
model.load_state_dict(model_weights)

# Get feature extractor
feature_extractor = torchvision.models.resnet18(pretrained=False)
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-2])
feature_extractor.to(device)
feature_extractor.load_state_dict(feature_extractor_weights)

<All keys matched successfully>

In [29]:
model.eval()
feature_extractor.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [30]:
def get_bounding_boxes(samples):
    # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
    # You need to return a tuple with size 'batch_size' and each element is a cuda tensor [N, 2, 4]
    # where N is the number of object

    #Preparing inputs
    batchsize = samples.shape[0]
    fe_batch = []
    for i in range(batchsize):
        image_tensor = sample[i]
        features = feature_extractor(image_tensor)
        #print(features.shape)
        features = concat_features(features)
        features = features.view(3, 512, 160)
        #print(features.shape)
        fe_batch.append(features)

    images = list(image.to(device) for image in fe_batch)
    predictions = model(images)
    res = []
    for i in range(len(predictions)):
        prediction = predictions[i]
        pred_bboxes = prediction['boxes']
        reorder_pred_bboxes = reorder_coord(pred_bboxes)
        res.append(reorder_pred_bboxes)

    return res

In [31]:
total = 0
total_ats_bounding_boxes = 0
total_ts_road_map = 0
for i, data in enumerate(dataloader):
    total += 1
    sample, target, road_image = data
    sample = sample.cuda()

    predicted_bounding_boxes = get_bounding_boxes(sample)[0].cpu()
    

    ats_bounding_boxes = compute_ats_bounding_boxes(predicted_bounding_boxes,
                                                    target['bounding_box'][0])
    print('Number of pred bboxes {}'.format(predicted_bounding_boxes.shape))
    print('ats_bounding_boxes {}'.format(ats_bounding_boxes))

    total_ats_bounding_boxes += ats_bounding_boxes


Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.008991344831883907
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.007450771518051624
Number of pred bboxes torch.Size([48, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([48, 2, 4])
ats_bounding_boxes 0.01605631224811077
Number of pred bboxes torch.Size([52, 2, 4])
ats_bounding_boxes 0.011781761422753334
Number of pred bboxes torch.Size([40, 2, 4])
ats_bounding_boxes 0.01092779915779829
Number of pred bboxes torch.Size([57, 2, 4])
ats_bounding_boxes 0.01138903759419918
Number of pred bboxes torch.Size([55, 2, 4])
ats_bounding_boxes 0.008478464558720589
Number of pred bboxes torch.Size([38, 2, 4])
ats_bounding_boxes 0.006542140617966652
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.007450771518051624
Number of pred bboxes torch.Size([60, 2, 4])
ats_bounding_boxes 0.004257583990693092
Number of pred bboxes torch.Size([57, 2, 4])
ats_bounding_boxes 0.01138903759419918
N

Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0161044392734766
Number of pred bboxes torch.Size([74, 2, 4])
ats_bounding_boxes 0.0053451182320714
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.006705694366246462
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.009707042016088963
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.003013795241713524
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.005588078871369362
Number of pred bboxes torch.Size([60, 2, 4])
ats_bounding_boxes 0.00887457374483347
Number of pred bboxes torch.Size([61, 2, 4])
ats_bounding_boxes 0.006470407359302044
Number of pred bboxes torch.Size([47, 2, 4])
ats_bounding_boxes 0.0078055704943835735
Number of pred bboxes torch.Size([44, 2, 4])
ats_bounding_boxes 0.008061490021646023
Number of pred bboxes torch.Size([60, 2, 4])
ats_bounding_boxes 0.006556679494678974
Number of pred bboxes torch.Size([56, 2, 4])
ats_bounding_boxes 0.023

Number of pred bboxes torch.Size([54, 2, 4])
ats_bounding_boxes 0.009762031957507133
Number of pred bboxes torch.Size([67, 2, 4])
ats_bounding_boxes 0.005854177754372358
Number of pred bboxes torch.Size([51, 2, 4])
ats_bounding_boxes 0.004126581363379955
Number of pred bboxes torch.Size([59, 2, 4])
ats_bounding_boxes 0.006556679494678974
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.008541777729988098
Number of pred bboxes torch.Size([77, 2, 4])
ats_bounding_boxes 0.007678002119064331
Number of pred bboxes torch.Size([50, 2, 4])
ats_bounding_boxes 0.004257583990693092
Number of pred bboxes torch.Size([75, 2, 4])
ats_bounding_boxes 0.003083077957853675
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([77, 2, 4])
ats_bounding_boxes 0.005588078871369362
Number of pred bboxes torch.Size([78, 2, 4])
ats_bounding_boxes 0.0053451182320714
Number of pred bboxes torch.Size([69, 2, 4])
ats_bounding_boxes 0.0
Number of pred b

Number of pred bboxes torch.Size([72, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([58, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([51, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([45, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([58, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([61, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([53, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([52, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([51, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([50, 2, 4])
ats_bounding_boxes 0.004546233452856541
Number of pred bboxes torch.Siz

Number of pred bboxes torch.Size([67, 2, 4])
ats_bounding_boxes 0.006736313924193382
Number of pred bboxes torch.Size([81, 2, 4])
ats_bounding_boxes 0.016113964840769768
Number of pred bboxes torch.Size([72, 2, 4])
ats_bounding_boxes 0.01652289554476738
Number of pred bboxes torch.Size([74, 2, 4])
ats_bounding_boxes 0.018917065113782883
Number of pred bboxes torch.Size([92, 2, 4])
ats_bounding_boxes 0.014642586931586266
Number of pred bboxes torch.Size([74, 2, 4])
ats_bounding_boxes 0.012627411633729935
Number of pred bboxes torch.Size([67, 2, 4])
ats_bounding_boxes 0.013290565460920334
Number of pred bboxes torch.Size([95, 2, 4])
ats_bounding_boxes 0.011502285487949848
Number of pred bboxes torch.Size([73, 2, 4])
ats_bounding_boxes 0.014302252791821957
Number of pred bboxes torch.Size([73, 2, 4])
ats_bounding_boxes 0.01593463309109211
Number of pred bboxes torch.Size([74, 2, 4])
ats_bounding_boxes 0.00860758125782013
Number of pred bboxes torch.Size([85, 2, 4])
ats_bounding_boxes 0.01

...

Number of pred bboxes torch.Size([67, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([61, 2, 4])
ats_bounding_boxes 0.009111229330301285
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.006470407359302044
Number of pred bboxes torch.Size([83, 2, 4])
ats_bounding_boxes 0.012094207108020782
Number of pred bboxes torch.Size([81, 2, 4])
ats_bounding_boxes 0.015102250501513481
Number of pred bboxes torch.Size([92, 2, 4])
ats_bounding_boxes 0.007520404644310474
Number of pred bboxes torch.Size([84, 2, 4])
ats_bounding_boxes 0.010341214947402477
Number of pred bboxes torch.Size([74, 2, 4])
ats_bounding_boxes 0.008416916243731976
Number of pred bboxes torch.Size([87, 2, 4])
ats_bounding_boxes 0.012772142887115479
Number of pred bboxes torch.Size([81, 2, 4])
ats_bounding_boxes 0.015902526676654816
Number of pred bboxes torch.Siz

Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.023171309381723404
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.024711912497878075
Number of pred bboxes torch.Size([56, 2, 4])
ats_bounding_boxes 0.034008413553237915
Number of pred bboxes torch.Size([70, 2, 4])
ats_bounding_boxes 0.026015089824795723
Number of pred bboxes torch.Size([74, 2, 4])
ats_bounding_boxes 0.030652672052383423
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.02699606493115425
Number of pred bboxes torch.Size([50, 2, 4])
ats_bounding_boxes 0.024583645164966583
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.00958461593836546
Number of pred bboxes torch.Size([52, 2, 4])
ats_bounding_boxes 0.018048454076051712
Number of pred bboxes torch.Size([77, 2, 4])
ats_bounding_boxes 0.026930350810289383
Number of pred bboxes torch.Size([59, 2, 4])
ats_bounding_boxes 0.04050463065505028
Number of pred bboxes torch.Size([68, 2, 4])
ats_bounding_boxes 0.03

Number of pred bboxes torch.Size([72, 2, 4])
ats_bounding_boxes 0.00773184560239315
Number of pred bboxes torch.Size([83, 2, 4])
ats_bounding_boxes 0.01066697295755148
Number of pred bboxes torch.Size([73, 2, 4])
ats_bounding_boxes 0.01173196081072092
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.014565514400601387
Number of pred bboxes torch.Size([89, 2, 4])
ats_bounding_boxes 0.009944772347807884
Number of pred bboxes torch.Size([69, 2, 4])
ats_bounding_boxes 0.00676576467230916
Number of pred bboxes torch.Size([85, 2, 4])
ats_bounding_boxes 0.008552188985049725
Number of pred bboxes torch.Size([83, 2, 4])
ats_bounding_boxes 0.005840532016009092
Number of pred bboxes torch.Size([85, 2, 4])
ats_bounding_boxes 0.007273372262716293
Number of pred bboxes torch.Size([67, 2, 4])
ats_bounding_boxes 0.009934362024068832
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.0130027299746871
Number of pred bboxes torch.Size([49, 2, 4])
ats_bounding_boxes 0.0065

Number of pred bboxes torch.Size([73, 2, 4])
ats_bounding_boxes 0.01826615072786808
Number of pred bboxes torch.Size([77, 2, 4])
ats_bounding_boxes 0.026930350810289383
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.016990669071674347
Number of pred bboxes torch.Size([69, 2, 4])
ats_bounding_boxes 0.013290565460920334
Number of pred bboxes torch.Size([72, 2, 4])
ats_bounding_boxes 0.01091005839407444
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.01587149128317833
Number of pred bboxes torch.Size([79, 2, 4])
ats_bounding_boxes 0.0031931877601891756
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.02286360412836075
Number of pred bboxes torch.Size([60, 2, 4])
ats_bounding_boxes 0.01035366952419281
Number of pred bboxes torch.Size([44, 2, 4])
ats_bounding_boxes 0.02439964935183525
Number of pred bboxes torch.Size([58, 2, 4])
ats_bounding_boxes 0.015241626650094986
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.0167

Number of pred bboxes torch.Size([70, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([61, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([69, 2, 4])
ats_bounding_boxes 0.0061468868516385555
Number of pred bboxes torch.Size([73, 2, 4])
ats_bounding_boxes 0.006463319528847933
Number of pred bboxes torch.Size([77, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.00352931278757751
Number of pred bboxes torch.Size([57, 2, 4])
ats_bounding_boxes 0.007339565549045801
Number of pred bboxes torch.Size([73, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.0111971665173769
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.006829873658716679
Number of pred bboxes torch.Size([49, 2, 4])
ats_bounding_boxes 0.011202330701053143
Number of pred bboxes torch.Size([68, 2, 4])
ats_bounding_boxes 0.00958461593836546
Number of pred bboxes torch.Size([76, 2, 4])
ats_bo

In [32]:
print(total_ats_bounding_boxes)
print(total_ats_bounding_boxes / total)

tensor(19.6162)
tensor(0.0111)


## fastRCNN - version of 05/01


In [5]:
def get_transform(): 
    return torchvision.transforms.ToTensor()


def extract_features(one_sample):
    feature_extractor = torchvision.models.resnet50(pretrained=False)
    feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-2])
    feature_extractor.cuda()
    return feature_extractor(one_sample)

def concat_features(features, dim = 2):
    #dim 0 ==> stacking the images in the channel dimension
    #dim 1 ==> stacking the images in row dimension
    #dim 2 ==> stacking the images in column dimension
    tensor_tuples = torch.unbind(features, dim=0)
    concatenated_fm = torch.cat(tensor_tuples, dim=dim)
    return concatenated_fm 

def prepare_inputs(samples):
    """
    Input: samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
    Output: a list of batch_size tensor, each tensor with size [512, 16, 114]
    """
    batchsize = samples.shape[0]
    fe_batch = []
    for i in range(batchsize):
        image_tensor = samples[i]
        features = extract_features(image_tensor)
        #print(features.shape)
        features = concat_features(features)
        features = features.view(3, 2048, 160)
        #print(features.shape)
        fe_batch.append(features)
    
    return fe_batch

# Need to convert this to 
def reorder_coord(pred_bboxes):
    xmin, ymin, xmax, ymax = pred_bboxes.unbind(1)
    return torch.stack((xmax, xmax, xmin, xmin, ymax, ymin, ymax, ymin), dim=1).view(-1, 2, 4)


In [6]:
class ModelLoader():
    # Fill the information for your team
    team_name = 'team_name'
    round_number = 1
    team_member = []
    contact_email = '@nyu.edu'

    def __init__(self, model_file='put_your_model_file_name_here'):
        """
        model_file = {'get_bboxes_model': object_detection_model_path,
                      'get_binary_RM_model': None}
        """
        self.bboxes_model_weights =  model_file['get_bboxes_model']
        self.bboxes_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        self.bboxes_model = self.bboxes_model.to(device)
        #self.model_path = '/scratch/nhl256/dl_project/model/object_detection_resnet18_epoch2.pth'
        self.bboxes_model.load_state_dict(torch.load(self.bboxes_model_weights))
        self.bboxes_model.eval()

    def get_bounding_boxes(self, samples):
        # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
        # You need to return a tuple with size 'batch_size' and each element is a cuda tensor [N, 2, 4]
        # where N is the number of object
        inputs = prepare_inputs(samples)
        predictions = self.bboxes_model(inputs)
        res = []
        for i in range(len(predictions)):
            prediction = predictions[i]
            pred_bboxes = prediction['boxes']
            reorder_pred_bboxes = reorder_coord(pred_bboxes)
            res.append(reorder_pred_bboxes)

        return res

    def get_binary_road_map(self, samples):
        # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
        # You need to return a cuda tensor with size [batch_size, 800, 800] 
        
        return torch.rand(1, 800, 800) > 0.5

In [7]:
object_detection_model_path = '/scratch/nhl256/dl_project/model/object_detection_resnet50_epoch5.pth'
model_file = {'get_bboxes_model': object_detection_model_path,
               'get_binary_RM_model': None}
model_loader = ModelLoader(model_file)

In [ ]:
total = 0
total_ats_bounding_boxes = 0
total_ts_road_map = 0
for i, data in enumerate(dataloader):
    total += 1
    sample, target, road_image = data
    sample = sample.cuda()

    predicted_bounding_boxes = model_loader.get_bounding_boxes(sample)[0].cpu()
    

    ats_bounding_boxes = compute_ats_bounding_boxes(predicted_bounding_boxes,
                                                    target['bounding_box'][0])
    print('Number of pred bboxes {}'.format(predicted_bounding_boxes.shape))
    print('ats_bounding_boxes {}'.format(ats_bounding_boxes))

    total_ats_bounding_boxes += ats_bounding_boxes

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.002554550301283598
Number of pred bboxes torch.Size([92, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([98, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.004774280358105898
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([86, 2, 4])
ats_bounding_boxes 0.003013795241713524
Number of pred bboxes torch.Size([93, 2, 4])
ats_bounding_boxes 0.002794039435684681
Number of pred bboxes torch.Size([77, 2, 4])
ats_bounding_boxes 0.0033114540856331587
Number of pred bboxes 

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.011750979349017143
Number of pred bboxes torch.Size([77, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.004276094492524862
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0023123084101825953
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0046648308634757996
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.004239232279360294
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.010029654018580914
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.005742372013628483
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.006424579303711653
Number of pred bboxes torch.Size([78, 2, 4])
ats_bounding_boxes 0.005646900739520788
Number of pred bbox

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0068286750465631485
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.004351777955889702
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0048329331912100315
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.002460805233567953
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.004967181012034416
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.004876868333667517
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.002460805233567953
Number of pred bboxes torch.Size([100, 2, 4])
ats_b

Number of pred bboxes torch.Size([89, 2, 4])
ats_bounding_boxes 0.008683515712618828
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([96, 2, 4])
ats_bounding_boxes 0.004728374537080526
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([97, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([87, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([97, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([88, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([97, 2, 4])
ats_bounding_boxes 0.0
Number 

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.010230286978185177
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.010230286978185177
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.00853785127401352
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.019020287320017815
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.00812811404466629
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.007709123659878969
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.00846753641963005
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.002198588103055954
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.011872241273522377
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.006265673786401749
Numbe

...

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.007078321184962988
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.009316553361713886
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.005659126210957766
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.003615815658122301
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.003697375301271677
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0020167501643300056
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.008990650065243244
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0019868724048137665
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.001972262980416417
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.005061793606728315
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.003915734123438597
Number of pred bboxes torch.Size([100, 2, 4])
ats_bo

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.002163127064704895
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([84, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([95, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bbox

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.004239232279360294
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([86, 2, 4])
ats_bounding_boxes 0.004868821240961552
Number of pred bboxes torch.Size([98, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([67, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.00958461593836546
Number of pred bboxes torch.Size([72, 2, 4])
ats_bounding_boxes 0.01348179206252098
Number of pred bboxes torch.Size([72, 2, 4])
ats_bounding_boxes 0.0029803086072206497
Number of pred bboxes torch.Size([68, 2, 4])
ats_bounding_boxes 0.008807087317109108
Number of pred bboxes torch.Size([80, 2, 4])
ats_bounding_boxes 0.0075028445571660995
Number of pred bboxes torch.Size([70, 2, 4])
ats_bounding_box

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0039979745633900166
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.011872241273522377
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0021458223927766085
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.007931466214358807
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.010892994701862335
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.009541040286421776
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0021458223927766085
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.003934007603675127
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0021458223927766085
Number of pred bboxes torch.Size([87, 2, 4])
ats_bounding_boxes 0.015122024342417717
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.008061490021646023
Number of pred bboxes torch.Size([100, 2, 4])
ats_b

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0020792849827557802
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.005923796445131302
Number of pred bboxes torch.Size([94, 2, 4])
ats_bounding_boxes 0.002163127064704895
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0040640574879944324
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.003697375301271677
Number of pred bboxes torch.Size([76, 2, 4])
ats_bounding_boxes 0.006212201435118914
Number of pred bboxes torch.Size([89, 2, 4])
ats_bounding_boxes 0.01138903759419918
Number of pred bboxes torch.Size([99, 2, 4])
ats_bounding_boxes 0.003725385759025812
Number of pred bboxes torch.Size([87, 2, 4])
ats_bounding_boxes 0.006370722781866789
Number of pred bboxes torch.Size([91, 2, 4])
ats_bounding_boxes 0.005423350725322962
Number of pred bboxes torch.Size([86, 2, 4])
ats_bounding_boxes 0.002198588103055954
Number of pred bboxes torch.Size([97, 2, 4])
ats_bounding_bo

In [10]:
total_ats_bounding_boxes

tensor(5.2924)

## fastRCNN - Version of 04/29/2020

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

import math
import time

import utils

In [6]:
def get_transform(): 
    return torchvision.transforms.ToTensor()


def extract_features(one_sample):
    feature_extractor = torchvision.models.resnet101(pretrained=False)
    feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-2])
    feature_extractor.to(device)
    return feature_extractor(one_sample)

def position_concat_features(feature_maps, dim):
    '''
    feature maps be in shape V C H W
    returns in shape C H W
    '''
    first_row = torch.cat([feature_maps[i] for i in range(3)], dim=dim)
    second_row = torch.cat([feature_maps[i] for i in range(3,6)], dim=dim)
    result = torch.cat([first_row, second_row], dim=1)
    return result

# Need to convert this to 
def reorder_coord(pred_bboxes):
    xmin, ymin, xmax, ymax = pred_bboxes.unbind(1)
    return torch.stack((xmax, xmax, xmin, xmin, ymax, ymin, ymax, ymin), dim=1).view(-1, 2, 4)


def prepare_inputs(sample):
    """
    Input: samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
    Output: a list of batch_size tensor, each tensor with size [512, 16, 114]
    """
    batchsize = sample.shape[0]
    fe_batch = []
    for i in range(batchsize):
        image_tensor = sample[i]
        features = extract_features(image_tensor)
        #print(features.shape)
        features = position_concat_features(features, 2)
        features = features.view(3, 2048, 160)
        #print(features.shape)
        fe_batch.append(features)
    
    return fe_batch


In [7]:
class ModelLoader():
    # Fill the information for your team
    team_name = 'team_name'
    round_number = 1
    team_member = []
    contact_email = '@nyu.edu'

    def __init__(self, model_file='put_your_model_file_name_here'):
        """
        model_file = {'get_bboxes_model': object_detection_model_path,
                      'get_binary_RM_model': None}
        """
        self.bboxes_model_weights =  model_file['get_bboxes_model']
        self.bboxes_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
        self.bboxes_model = self.bboxes_model.to(device)
        self.bboxes_model.load_state_dict(torch.load(self.bboxes_model_weights))
        self.bboxes_model.eval()
    
    def get_bounding_boxes(self, samples):
        # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
        # You need to return a tuple with size 'batch_size' and each element is a cuda tensor [N, 2, 4]
        # where N is the number of object
        inputs = prepare_inputs(samples)
        predictions = self.bboxes_model(inputs)
        res = []
        for i in range(len(predictions)):
            prediction = predictions[i]
            pred_bboxes = prediction['boxes']
            reorder_pred_bboxes = reorder_coord(pred_bboxes)
            res.append(reorder_pred_bboxes)

        return res

    def get_binary_road_map(self, samples):
        # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
        # You need to return a cuda tensor with size [batch_size, 800, 800] 
        
        return torch.rand(1, 800, 800) > 0.5

In [8]:
# GET MODEL
object_detection_model_path = '/scratch/nhl256/dl_project/model/obDect_resnet101_epoch1.pth'
model_file = {'get_bboxes_model': object_detection_model_path,
               'get_binary_RM_model': None}
model_loader = ModelLoader(model_file)
model = model_loader.bboxes_model

In [ ]:
total = 0
total_ats_bounding_boxes = 0
total_ts_road_map = 0
for i, data in enumerate(dataloader):
    total += 1
    sample, target, road_image = data
    sample = sample.cuda()

    predicted_bounding_boxes = model_loader.get_bounding_boxes(sample)[0].cpu()
    

    ats_bounding_boxes = compute_ats_bounding_boxes(predicted_bounding_boxes,
                                                    target['bounding_box'][0])
    print('Number of pred bboxes {}'.format(predicted_bounding_boxes.shape))
    print('ats_bounding_boxes {}'.format(ats_bounding_boxes))

    total_ats_bounding_boxes += ats_bounding_boxes

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([

...

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([93, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([9

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([

Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([100, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([

In [ ]:
print(total_ats_bounding_boxes)

## fastRCNN - Version of 04/20/2020

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

import math
import time

import utils

In [47]:
def get_transform(): 
    return torchvision.transforms.ToTensor()


def extract_features(one_sample):
    feature_extractor = torchvision.models.resnet18(pretrained=False)
    feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-2])
    feature_extractor.cuda()
    return feature_extractor(one_sample)

def concat_features(features, dim = 2):
    #dim 0 ==> stacking the images in the channel dimension
    #dim 1 ==> stacking the images in row dimension
    #dim 2 ==> stacking the images in column dimension
    tensor_tuples = torch.unbind(features, dim=0)
    concatenated_fm = torch.cat(tensor_tuples, dim=dim)
    return concatenated_fm 

def prepare_inputs(samples):
    """
    Input: samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
    Output: a list of batch_size tensor, each tensor with size [512, 16, 114]
    """
    batchsize = samples.shape[0]
    fe_batch = []
    for i in range(batchsize):
        image_tensor = samples[i]
        features = extract_features(image_tensor)
        #print(features.shape)
        features = concat_features(features)
        features = features.view(3, 512, 160)
        #print(features.shape)
        fe_batch.append(features)
    
    return fe_batch

# Need to convert this to 
def reorder_coord(pred_bboxes):
    xmin, ymin, xmax, ymax = pred_bboxes.unbind(1)
    return torch.stack((xmax, xmax, xmin, xmin, ymax, ymin, ymax, ymin), dim=1).view(-1, 2, 4)


In [48]:
class ModelLoader():
    # Fill the information for your team
    team_name = 'team_name'
    round_number = 1
    team_member = []
    contact_email = '@nyu.edu'

    def __init__(self, model_file='put_your_model_file_name_here'):
        """
        model_file = {'get_bboxes_model': object_detection_model_path,
                      'get_binary_RM_model': None}
        """
        self.bboxes_model_weights =  model_file['get_bboxes_model']
        self.bboxes_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        self.bboxes_model = self.bboxes_model.to(device)
        #self.model_path = '/scratch/nhl256/dl_project/model/object_detection_resnet18_epoch2.pth'
        self.bboxes_model.load_state_dict(torch.load(self.bboxes_model_weights))
        self.bboxes_model.eval()

    def get_bounding_boxes(self, samples):
        # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
        # You need to return a tuple with size 'batch_size' and each element is a cuda tensor [N, 2, 4]
        # where N is the number of object
        inputs = prepare_inputs(samples)
        predictions = self.bboxes_model(inputs)
        res = []
        for i in range(len(predictions)):
            prediction = predictions[i]
            pred_bboxes = prediction['boxes']
            reorder_pred_bboxes = reorder_coord(pred_bboxes)
            res.append(reorder_pred_bboxes)

        return res

    def get_binary_road_map(self, samples):
        # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
        # You need to return a cuda tensor with size [batch_size, 800, 800] 
        
        return torch.rand(1, 800, 800) > 0.5

In [49]:
object_detection_model_path = '/scratch/nhl256/dl_project/model/object_detection_resnet18_epoch2.pth'

In [50]:
model_file = {'get_bboxes_model': object_detection_model_path,
               'get_binary_RM_model': None}

In [51]:
model_loader = ModelLoader(model_file)

In [52]:
print('number of samples in dataloader is {}'.format(dataloader.__len__()))

number of samples in dataloader is 1764


In [53]:
total = 0
total_ats_bounding_boxes = 0
total_ts_road_map = 0
for i, data in enumerate(dataloader):
    total += 1
    sample, target, road_image = data
    sample = sample.cuda()

    predicted_bounding_boxes = model_loader.get_bounding_boxes(sample)[0].cpu()
    

    ats_bounding_boxes = compute_ats_bounding_boxes(predicted_bounding_boxes,
                                                    target['bounding_box'][0])
    print('Number of pred bboxes {}'.format(predicted_bounding_boxes.shape))
    print('ats_bounding_boxes {}'.format(ats_bounding_boxes))

    total_ats_bounding_boxes += ats_bounding_boxes

Number of pred bboxes torch.Size([61, 2, 4])
ats_bounding_boxes 0.01035366952419281
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.007339565549045801
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.009762031957507133
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.0111971665173769
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.014492752961814404
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.007231631316244602
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.01035366952419281
Number of pred bboxes torch.Size([72, 2, 4])
ats_bounding_boxes 0.006556679494678974
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.010049150325357914
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.003944525960832834
Number of pred bboxes torch.Size([60, 2, 4])
ats_bounding_boxes 0.013507691211998463
Number of pred bboxes torch.Size([70, 2, 4])
ats_bounding_boxes 0.006

Number of pred bboxes torch.Size([73, 2, 4])
ats_bounding_boxes 0.019050292670726776
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.006463319528847933
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.006224695593118668
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.0033114540856331587
Number of pred bboxes torch.Size([60, 2, 4])
ats_bounding_boxes 0.009832636453211308
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.014302252791821957
Number of pred bboxes torch.Size([70, 2, 4])
ats_bounding_boxes 0.011436067521572113
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.0089103477075696
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.018913498148322105
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.015034204348921776
Number of pred bboxes torch.Size([59, 2, 4])
ats_bounding_boxes 0.012958360835909843
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.

Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.006070998962968588
Number of pred bboxes torch.Size([69, 2, 4])
ats_bounding_boxes 0.0057180337607860565
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.008760797791182995
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.005652309395372868
Number of pred bboxes torch.Size([78, 2, 4])
ats_bounding_boxes 0.00895773246884346
Number of pred bboxes torch.Size([58, 2, 4])
ats_bounding_boxes 0.006829873658716679
Number of pred bboxes torch.Size([61, 2, 4])
ats_bounding_boxes 0.006386375054717064
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.006224695593118668
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.00352931278757751
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([61, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([68, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size

Number of pred bboxes torch.Size([69, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([74, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([78, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([59, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([67, 2, 4])
ats

Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.013510332442820072
Number of pred bboxes torch.Size([68, 2, 4])
ats_bounding_boxes 0.013472627848386765
Number of pred bboxes torch.Size([60, 2, 4])
ats_bounding_boxes 0.01370492484420538
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.01022970862686634
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.009490863420069218
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.0070250132121145725
Number of pred bboxes torch.Size([61, 2, 4])
ats_bounding_boxes 0.009762031957507133
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.0070250132121145725
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.013321190141141415
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.014820592477917671
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.00887457374483347
Number of pred bboxes torch.Size([70, 2, 4])
ats_bounding_boxes 0.

...

Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([61, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([68, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([72, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([58, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([70, 2, 4])
ats

Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.025700083002448082
Number of pred bboxes torch.Size([68, 2, 4])
ats_bounding_boxes 0.034742917865514755
Number of pred bboxes torch.Size([58, 2, 4])
ats_bounding_boxes 0.036918286234140396
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.022821558639407158
Number of pred bboxes torch.Size([67, 2, 4])
ats_bounding_boxes 0.017687756568193436
Number of pred bboxes torch.Size([68, 2, 4])
ats_bounding_boxes 0.016867274418473244
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.015577632002532482
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.022273819893598557
Number of pred bboxes torch.Size([61, 2, 4])
ats_bounding_boxes 0.020729055628180504
Number of pred bboxes torch.Size([70, 2, 4])
ats_bounding_boxes 0.01970849372446537
Number of pred bboxes torch.Size([70, 2, 4])
ats_bounding_boxes 0.021558498963713646
Number of pred bboxes torch.Size([60, 2, 4])
ats_bounding_boxes 0.

Number of pred bboxes torch.Size([67, 2, 4])
ats_bounding_boxes 0.0049175089225173
Number of pred bboxes torch.Size([76, 2, 4])
ats_bounding_boxes 0.0024384341668337584
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.00273701804690063
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([76, 2, 4])
ats_bounding_boxes 0.002506801625713706
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.002823450369760394
Number of pred bboxes torch.Size([59, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([61, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([68, 2, 4])
ats_bounding_boxes 0.005311441142112017
Number of pred bboxes torch.Size([60, 2, 4])
ats_bounding_boxes 0.002794039435684681
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.0

Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.02466621808707714
Number of pred bboxes torch.Size([67, 2, 4])
ats_bounding_boxes 0.013852139003574848
Number of pred bboxes torch.Size([75, 2, 4])
ats_bounding_boxes 0.01484428159892559
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.015429012477397919
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.016990669071674347
Number of pred bboxes torch.Size([71, 2, 4])
ats_bounding_boxes 0.020428279414772987
Number of pred bboxes torch.Size([73, 2, 4])
ats_bounding_boxes 0.01737830974161625
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.01370492484420538
Number of pred bboxes torch.Size([70, 2, 4])
ats_bounding_boxes 0.023000864312052727
Number of pred bboxes torch.Size([75, 2, 4])
ats_bounding_boxes 0.01484428159892559
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.01858440786600113
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.02310

Number of pred bboxes torch.Size([69, 2, 4])
ats_bounding_boxes 0.006224695593118668
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.00352931278757751
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.003395288484171033
Number of pred bboxes torch.Size([69, 2, 4])
ats_bounding_boxes 0.006542140617966652
Number of pred bboxes torch.Size([67, 2, 4])
ats_bounding_boxes 0.003271070308983326
Number of pred bboxes torch.Size([67, 2, 4])
ats_bounding_boxes 0.00958461593836546
Number of pred bboxes torch.Size([72, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([66, 2, 4])
ats_bounding_boxes 0.01934334821999073
Number of pred bboxes torch.Size([64, 2, 4])
ats_bounding_boxes 0.024071721360087395
Number of pred bboxes torch.Size([69, 2, 4])
ats_bounding_boxes 0.029198117554187775
Number of pred bboxes torch.Size([

In [54]:
total_ats_bounding_boxes 

tensor(19.2864)